# Working with MPAS Meshes

##  Mesh Definition

All components of MPAS lie on a C-grid staggered Voronoi Mesh.

## Imports

In [ ]:
import xarray as xr
import uxarray as ux

## Grid and Data Paths

In [ ]:
mpas_root_filepath = "../../test/meshfiles/mpas/"
mpas_dataset_filepath = mpas_root_filepath + "ice-sheet/gis_20km_r01.20210824.nc"

# Constructing a Grid Object

In [ ]:
mpas_xr_ds = xr.open_dataset(mpas_dataset_filepath)
mpas_xr_ds

In [ ]:
primal_grid = ux.Grid(mpas_xr_ds, use_dual=False)
dual_grid = ux.Grid(mpas_xr_ds, use_dual=True)

In [ ]:
primal_grid.ds

In [ ]:
dual_grid.ds

## References

[1] https://mpas-dev.github.io/
[2] https://mpas-dev.github.io/files/documents/MPAS-MeshSpec.pdf